In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/working'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import os
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.backends import cudnn #optimizes runtime ??
import torch.optim as optim

# **Sample Tensor**

In [ ]:
x = torch.rand(2,1,3,3)
size=torch.flatten(x, start_dim=1)
print(x)
new = x.view(-1) # -1 here will be 2. because 18/(1*3*3)=2
print(new)
print(size)

In [ ]:


import zipfile

Dataset = "train"

# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/dogs-vs-cats/"+Dataset+".zip","r") as z:
    z.extractall(".")

In [ ]:
# more advanced options like dropout/batch normalization or additional transoformation techniques (random flip)
# can be added to overcome overfitting
# generally we can add as many layers as we want until we see overfitting in testing data and we can overcome it with
# above mentioned methods
class my_net(nn.Module):
    def __init__(self):
        super(my_net, self).__init__()
        self.conv1 = nn.Conv2d(3,50,5)
        self.conv2 = nn.Conv2d(50,150,3)
        self.conv3 = nn.Conv2d(150,300,3)
        self.conv4 = nn.Conv2d(300, 120, 3)
        self.fc1 = nn.Linear(3000, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear (512, 2)
    
        
    def forward(self,x): 
        # when you simply do "net(sample)" without calling this method explicitly, it is called because of __call__ method in nn.Module, 
        # you cannot replace name of this method because it inherits from nn.module
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 3, stride = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 3, stride = 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, kernel_size = 3, stride = 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, kernel_size = 3, stride = 2)
        x=torch.flatten(x, start_dim=1)
        #print("Required shape: ", x.shape) # to decide the input dimension of fc1
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = my_net()
print(net)

In [ ]:
sample = torch.rand(1,3,128,128)
output = net(sample)
output.shape

In [ ]:

os.getcwd()

In [ ]:
train_transform = transforms.Compose([
                                      transforms.Resize(256),
                                      #transforms.RandomCrop( 64 , padding =2) ,
                                      #transforms.RandomHorizontalFlip(),
                                      transforms.RandomCrop(192), # if we does not do that we will have samples with different input sizes 
                                                                  # which is not accepted by cnn
                                      transforms.Resize(128),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
                                      # Normalizes tensor with mean and standard deviation
])

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
class MyCatsDogs(Dataset):
    def __init__(self, root, transform = None, mode = 'train'):
        super(MyCatsDogs, self).__init__()
        self.root = root
        self.transform = transform
        self.mode = mode
        self.images = []
        self.labels = []
        
        for image in os.listdir(self.root):
            if(image.split('.')[0].lower() == 'cat'):
                self.labels.append(1)
            else:
                self.labels.append(0)
            
            self.images.append(image)
    
    
    
    def __len__(self):
        return len(self.images)
    
    
        
    def split(self, start, end): # Checking if I take 20% of the data from the end for validation 
                                 # will it be balanced data?kagg
        return self.labels[ start : end+1]
    
    
    
    def show_image(self, index): # for visualizing the image
        
        filename = os.path.join(self.root, self.images[index])
        img_array = np.array(Image.open(filename))
        plt.imshow(img_array)

        
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.root, self.images[index]))
        
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = np.array(img)
            img = img.astype('float32')
        
        return img, self.labels[index]

In [ ]:
train_dir = './train'
sample_object = MyCatsDogs(root=train_dir, transform = train_transform)  
my_array = sample_object.__getitem__(1)[0] # for returning label of the 0th element
print("result: ", my_array.shape)
my_split = sample_object.split(20001,25000)
print("Split: ", sum(my_split)) # = 2510 so balanced distribution

sample  = sample_object.show_image(0)
len(sample_object)

In [ ]:
train_indexes =  [idx for idx in range(20000)]
valid_indexes =  [idx for idx in range(20000, 25000)]

train_data = Subset(sample_object, train_indexes)
valid_data  = Subset(sample_object, valid_indexes)

print(len(train_data), len(valid_data))

In [ ]:
train_loader = DataLoader(train_data, batch_size = 64, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_data, batch_size = 64, shuffle=True, num_workers=4)

In [ ]:
net = net.to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones = [3, 7], gamma=0.1)
epochs = 10

In [ ]:
cudnn.benchmark # Calling this optimizes runtime
current_step = 0
loss_values = []

# Start iterating over the epochs
for epoch in range(epochs):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, epochs, scheduler.get_lr()))
    
    # Iterate over the dataset
    for images, labels in tqdm(train_loader):
        
    # Bring data over the device of choice
        images = images.to('cuda')
        labels = labels.to('cuda')
    
        net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
        optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
        outputs = net(images)
    
    # Compute loss based on output and ground truth
        loss = criterion(outputs, labels)

    # Log loss
        if current_step % 100 == 0:
            loss_values.append(loss.item())
            #step_values.append(current_step)
            print('Step {}, Loss {} , Lr {}'.format(current_step, loss.item(), scheduler.get_lr()))
            
      

    # Compute gradients for each layer and update weights
        loss.backward()  # backward pass: computes gradients
        optimizer.step() # update weights based on accumulated gradients

        current_step += 1
    
    # Step the scheduler
    scheduler.step()

    steps = [x for x in range(0, 100*len(loss_values), 100)]
    plt.plot(steps, loss_values)
    plt.show()

In [ ]:
steps = [x for x in range(0, 100*len(loss_values), 100)]

plt.plot(steps, loss_values)
plt.show()

In [ ]:
net = net.to('cuda') # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(valid_loader):
    
    
    images = images.to('cuda')
    labels = labels.to('cuda')

  # Forward Pass
    outputs = net(images)

  # Get predictions
    _, preds = torch.max(outputs.data, 1)

  # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(valid_data))

print('Test Accuracy: {}'.format(accuracy))